<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/mbp_mins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculation of MINS risk depending on BP during surgery
In this example, we will calculate the risk of MINS(myocardial injury after non-cardiac surgery, MINS) depending on blood pressure during surgery.

> Note that <b>all users who use Vital DB, an open biosignal dataset, must agree to the Data Use Agreement below. 
</b> If you do not agree, please close this window. 
Click here: [Data Use Agreement](https://vitaldb.net/dataset/?query=overview&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.vcpgs1yemdb5)

## Required libraries and datasets

In [1]:
import pandas as pd
import numpy as np

df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # 임상 정보
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # 트랙 목록
df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # lab 데이터

## Case Selection
Use only 100 patients with troponin I results.

In [2]:
caseids = list(
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']['caseid']) & 
    set(df_labs[df_labs['name'] == 'Troponin I']['caseid'])
)
caseids = caseids[:100]
print('Total {} cases found'.format(len(caseids)))

Total 100 cases found


## Load and calculate data for each case
- First of all, calculate the postoperative troponin I concentration for each case from vitaldb, and determine whether MINS occurs from this. In this example, MINS is defined as troponin I > 0.028 ng/mL.
- Also, let's get the ART_MBP data during surgery and find the ratio of the measured values that stayed below the threshold during the entire operation. The threshold range is 40-80mmHg.

In [ ]:
# Set blood pressure threshold
mbp_thresholds = np.arange(40, 80)

# Save the final result
df = pd.DataFrame()
for caseid in caseids:
    print('loading {}...'.format(caseid), flush=True, end='')

    # Column ['anend'] : anesthesia end time
    aneend = df_cases[(df_cases['caseid'] == caseid)]['aneend'].values[0]

    # Maximum creatinine concentration within 48 hours after surgery
    postop_tpi = df_labs[(df_labs['caseid'] == caseid) & (df_labs['dt'] > aneend) &
        (df_labs['dt'] < aneend + 48 * 3600) & (df_labs['name'] == 'Troponin I')]['result'].max(skipna=True)
    if not postop_tpi or np.isnan(postop_tpi):
        print('no postop troponin I')
        continue

    # mins = postop_tpi > 0.028
    mins = postop_tpi > 0.028

    # Blood pressure during surgery
    tid_mbp = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Solar8000/ART_MBP')]['tid'].values[0]
    mbps = pd.read_csv('https://api.vitaldb.net/' + tid_mbp).values[:,1]
    mbps = mbps[~np.isnan(mbps)]
    mbps = mbps[(mbps > 20) & (mbps < 150)]
    if len(mbps) < 10:
        print('no mbp')
        continue

    # Calculate the percentage that stays for the time as increasing the blood pressure by 1 unit.
    row = {'mins':mins}
    for mbp_threshold in mbp_thresholds:
        row['under{}'.format(mbp_threshold)] = np.nanmean(mbps < mbp_threshold) * 100

    # Append the result into row
    df = df.append(row, ignore_index=True)

    print('{}, {}'.format(postop_tpi, 'MINS' if mins else ''))

print('{} MINS {:.1f}%'.format(df['mins'].sum(), df['mins'].mean() * 100))

loading 1...0.01, 
loading 4...0.01, 
loading 6152...0.1, MINS
loading 10...0.69, MINS
loading 2058...0.01, 
loading 12...0.07, MINS
loading 2060...0.01, 
loading 4109...no postop troponin I
loading 2063...0.02, 
loading 2064...0.01, 
loading 17...0.01, 
loading 6156...0.01, 
loading 19...no postop troponin I
loading 20...0.02, 
loading 4115...0.03, MINS
loading 6159...no postop troponin I
loading 6166...0.01, 
loading 25...0.02, 
loading 6174...0.01, 
loading 4128...0.01, 
loading 2082...0.01, 
loading 6180...0.01, 
loading 2085...no postop troponin I
loading 4135...0.02, 
loading 6185...no postop troponin I
loading 6186...0.01, 
loading 4143...0.01, 
loading 2097...no postop troponin I
loading 4146...0.01, 
loading 2100...0.01, 
loading 4148...

## Calculate the odd ratio for MINS prediction depending on BP threshold

Let's calculate the odd ratio regarding how much each BP case increases the risk of MINS, using the above-measured values

In [ ]:
# Get odd ration using univariate logistic regression
import statsmodels.api as sm
df['intercept'] = 1
df['mins'] = df['mins'].astype(bool)
odd_ratios = []
for mbp_threshold in mbp_thresholds:
    c = 'under{}'.format(mbp_threshold)
    model = sm.Logit(df['mins'], df[['intercept', c]])
    res = model.fit()
    b = res.params[c]
    pval = res.pvalues[c]
    odd_ratios.append(np.exp(b))
    print('{}\tb={:.3f}, exp(b)={:.3f}, pval={:.3f}'.format(c, b, np.exp(b), pval))

## Draw a graph
We can find out that the odd ratio of MINS increases as the time that MBP stays below increases.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mbp_thresholds, odd_ratios)
plt.show()